Сначала напишем правила для каждого предложения по отдельности

In [39]:
sentence1 = 'Вася читает мою книгу'
rules1 = """
S -> DP VP
DP -> NProp
VP -> V DP
DP -> D NP
NP -> APRO NP
NP -> AP N | N
V -> 'читает'
NProp -> 'вася'
N -> 'книгу'
D -> 
APRO -> 'мою' 
"""

In [40]:
sentence2 = 'Напиши какое-нибудь письмо'
rules2 = """
S -> DP VP
DP -> 
VP -> V DP
DP -> D NP
NP -> AP N
AP -> 
V -> 'напиши'
D -> 'какое-нибудь'
N -> 'письмо'
"""

In [41]:
sentence3 = 'Этот веселый мальчик идет'
rules3 = """
S -> DP VP
VP -> V
DP -> D NP
NP -> AP N
AP -> A
A -> 'веселый'
N -> 'мальчик'
V -> 'идет'
D -> 'этот'
"""

In [42]:
sentence4 = 'Он любит читать всякие книги'
rules4 = """
S -> DP VP
DP -> SPRO
VP -> V VP
VP -> V DP
DP -> D NP
NP -> AP N
AP -> 
V -> 'любит' | 'читать'
D -> 'всякие'
N -> 'книги'
SPRO -> 'он'
"""

Теперь соберём их вместе:

In [43]:
common_rules = []
for indiv in [rules1, rules2, rules3, rules4]:
    for rule in indiv.split('\n'):
        if rule not in common_rules:
            common_rules.append(rule)
print('\n'.join(common_rules))


S -> DP VP
DP -> NProp
VP -> V DP
DP -> D NP
NP -> APRO NP
NP -> AP N | N
V -> 'читает'
NProp -> 'вася'
N -> 'книгу'
D -> 
APRO -> 'мою' 
DP -> 
NP -> AP N
AP -> 
V -> 'напиши'
D -> 'какое-нибудь'
N -> 'письмо'
VP -> V
AP -> A
A -> 'веселый'
N -> 'мальчик'
V -> 'идет'
D -> 'этот'
DP -> SPRO
VP -> V VP
V -> 'любит' | 'читать'
D -> 'всякие'
N -> 'книги'
SPRO -> 'он'


In [44]:
import nltk

In [45]:
grammar = nltk.CFG.fromstring('\n'.join(common_rules))

In [46]:
print(grammar)

Grammar with 31 productions (start state = S)
    S -> DP VP
    DP -> NProp
    VP -> V DP
    DP -> D NP
    NP -> APRO NP
    NP -> AP N
    NP -> N
    V -> 'читает'
    NProp -> 'вася'
    N -> 'книгу'
    D -> 
    APRO -> 'мою'
    DP -> 
    NP -> AP N
    AP -> 
    V -> 'напиши'
    D -> 'какое-нибудь'
    N -> 'письмо'
    VP -> V
    AP -> A
    A -> 'веселый'
    N -> 'мальчик'
    V -> 'идет'
    D -> 'этот'
    DP -> SPRO
    VP -> V VP
    V -> 'любит'
    V -> 'читать'
    D -> 'всякие'
    N -> 'книги'
    SPRO -> 'он'


In [47]:
parser = nltk.EarleyChartParser(grammar)

проверим, что оно рисует

In [48]:
for tree in parser.parse('Напиши всякие книги'.lower().split()):
    tree.draw()

Добавим возможность включать новые слова в разборы:

In [11]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [12]:
rules_list = str(grammar).split('\n')[1:]

In [13]:
good_rules_list = []
for item in rules_list:
    good_rules_list.append(item.strip(' '))

In [14]:
good_rules_list

['S -> DP VP',
 'DP -> NProp',
 'VP -> V DP',
 'DP -> D NP',
 'NP -> APRO NP',
 'NP -> AP N',
 "V -> 'читает'",
 "NProp -> 'вася'",
 "N -> 'книгу'",
 'AP ->',
 'D ->',
 "APRO -> 'мою'",
 'DP ->',
 "V -> 'напиши'",
 "D -> 'какое-нибудь'",
 "N -> 'письмо'",
 'VP -> V',
 'AP -> A',
 "A -> 'веселый'",
 "N -> 'мальчик'",
 "V -> 'идет'",
 "D -> 'этот'",
 'DP -> SPRO',
 'VP -> V VP',
 "V -> 'любит'",
 "V -> 'читать'",
 "D -> 'всякие'",
 "N -> 'книги'",
 "SPRO -> 'он'"]

In [15]:
import re

In [16]:
str(morph.parse('читает')[0].tag).split(',')[0]

'VERB'

In [17]:
def is_terminal(list_of_rules):
    new_rules = []
    for rule in list_of_rules:
        if re.search("'(.+?)'", rule):
            rule = rule.split(' -> ')
            pos = str(morph.parse(rule[1].strip("'"))[0].tag).split(',')[0]
            rule1 = ' -> '.join([rule[0], pos])
            rule2 = ' -> '.join([pos, rule[1]])
            new_rules.append(rule1)
            new_rules.append(rule2)
        else:
            new_rules.append(rule)
    return new_rules

In [18]:
widened_rules = '\n'.join(is_terminal(good_rules_list))

In [19]:
widened_rules

"S -> DP VP\nDP -> NProp\nVP -> V DP\nDP -> D NP\nNP -> APRO NP\nNP -> AP N\nV -> VERB\nVERB -> 'читает'\nNProp -> NOUN\nNOUN -> 'вася'\nN -> NOUN\nNOUN -> 'книгу'\nAP ->\nD ->\nAPRO -> ADJF\nADJF -> 'мою'\nDP ->\nV -> VERB\nVERB -> 'напиши'\nD -> ADJF\nADJF -> 'какое-нибудь'\nN -> NOUN\nNOUN -> 'письмо'\nVP -> V\nAP -> A\nA -> ADJF\nADJF -> 'веселый'\nN -> NOUN\nNOUN -> 'мальчик'\nV -> VERB\nVERB -> 'идет'\nD -> ADJF\nADJF -> 'этот'\nDP -> SPRO\nVP -> V VP\nV -> VERB\nVERB -> 'любит'\nV -> INFN\nINFN -> 'читать'\nD -> ADJF\nADJF -> 'всякие'\nN -> NOUN\nNOUN -> 'книги'\nSPRO -> NPRO\nNPRO -> 'он'"

In [20]:
new_grammar = nltk.CFG.fromstring(widened_rules)

In [21]:
#parser2 = nltk.EarleyChartParser(new_grammar)
#for tree in parser2.parse('Напиши всякие журналы'.lower().split()):
#    tree.draw()

In [22]:
def is_in_grammar(sentence):
    new_words = []
    global widened_rules
    for word in sentence.lower().split():
        if not re.search("'{}'".format(word), widened_rules):
            new_words.append(' -> '.join([str(morph.parse(word)[0].tag).split(',')[0], "'{}'".format(word)]))
        if new_words:
            widened_rules = widened_rules + '\n' + '\n'.join(new_words)
    new_grammar = nltk.CFG.fromstring(widened_rules)
    parser = nltk.EarleyChartParser(new_grammar)
    return parser

In [23]:
def draw_a_tree(sentence):
    parser2 = is_in_grammar(sentence)
    for tree in parser2.parse(sentence.lower().split()):
        tree.draw()

In [24]:
draw_a_tree('Умный Вася поет какие-то странные песни')

По-хорошему, можно оставить в основных правилах только переходы нетерминалов-групп в части речи, а уже их прописывать каждый раз свои в draw_a_tree (???)

In [25]:
shortened_rules = []
for rule in widened_rules.split('\n'):
    if not re.search("'(.+?)'", rule):
        shortened_rules.append(rule)
shortened_rules = '\n'.join(shortened_rules)

In [26]:
def add_terms_to_grammar(sentence):
    new_words = []
    for word in sentence.lower().split():
        if not re.search("'{}'".format(word), shortened_rules):
            new_words.append(' -> '.join([str(morph.parse(word)[0].tag).split(',')[0], "'{}'".format(word)]))
    if new_words:
        rules = shortened_rules + '\n' + '\n'.join(new_words)
    new_grammar = nltk.CFG.fromstring(rules)
    parser = nltk.EarleyChartParser(new_grammar)
    return parser

In [27]:
def tree_all_new_words(sentence):
    parser2 = add_terms_to_grammar(sentence)
    for tree in parser2.parse(sentence.lower().split()):
        print(tree)

In [28]:
tree_all_new_words('Кот ест всяких плохих мышей')

(S
  (DP (NProp (NOUN кот)))
  (VP
    (V (VERB ест))
    (DP
      (D (ADJF всяких))
      (NP (AP (A (ADJF плохих))) (N (NOUN мышей))))))
(S
  (DP (NProp (NOUN кот)))
  (VP
    (V (VERB ест))
    (DP
      (D )
      (NP
        (APRO (ADJF всяких))
        (NP (AP (A (ADJF плохих))) (N (NOUN мышей)))))))
(S
  (DP (NProp (NOUN кот)))
  (VP
    (V (VERB ест))
    (DP
      (D )
      (NP
        (APRO (ADJF всяких))
        (NP (APRO (ADJF плохих)) (NP (AP ) (N (NOUN мышей))))))))
(S
  (DP (NProp (NOUN кот)))
  (VP
    (V (VERB ест))
    (DP
      (D (ADJF всяких))
      (NP (APRO (ADJF плохих)) (NP (AP ) (N (NOUN мышей)))))))
(S
  (DP (D ) (NP (AP ) (N (NOUN кот))))
  (VP
    (V (VERB ест))
    (DP
      (D (ADJF всяких))
      (NP (AP (A (ADJF плохих))) (N (NOUN мышей))))))
(S
  (DP (D ) (NP (AP ) (N (NOUN кот))))
  (VP
    (V (VERB ест))
    (DP
      (D )
      (NP
        (APRO (ADJF всяких))
        (NP (AP (A (ADJF плохих))) (N (NOUN мышей)))))))
(S
  (DP (D ) (NP (AP ) (N (NOU

In [29]:
tree_all_new_words('мальчик пишет рассказы')

(S
  (DP (NProp (NOUN мальчик)))
  (VP (V (VERB пишет)) (DP (NProp (NOUN рассказы)))))
(S
  (DP (NProp (NOUN мальчик)))
  (VP (V (VERB пишет)) (DP (D ) (NP (AP ) (N (NOUN рассказы))))))
(S
  (DP (D ) (NP (AP ) (N (NOUN мальчик))))
  (VP (V (VERB пишет)) (DP (NProp (NOUN рассказы)))))
(S
  (DP (D ) (NP (AP ) (N (NOUN мальчик))))
  (VP (V (VERB пишет)) (DP (D ) (NP (AP ) (N (NOUN рассказы))))))


4-8 деревьев выглядят страшно, поэтому постараемся минимизировать переходы.

In [30]:
print(shortened_rules)

S -> DP VP
DP -> NProp
VP -> V DP
DP -> D NP
NP -> APRO NP
NP -> AP N
V -> VERB
NProp -> NOUN
N -> NOUN
AP ->
D ->
APRO -> ADJF
DP ->
V -> VERB
D -> ADJF
N -> NOUN
VP -> V
AP -> A
A -> ADJF
N -> NOUN
V -> VERB
D -> ADJF
DP -> SPRO
VP -> V VP
V -> VERB
V -> INFN
D -> ADJF
N -> NOUN
SPRO -> NPRO


In [31]:
how_many_rules = {}
for rule in shortened_rules.split('\n'):
    rule = rule.split('->')
    if rule[0] not in how_many_rules:
        how_many_rules[rule[0]] = [rule[1]]
    else:
        how_many_rules[rule[0]].append(rule[1])
how_many_rules

{'S ': [' DP VP'],
 'DP ': [' NProp', ' D NP', '', ' SPRO'],
 'VP ': [' V DP', ' V', ' V VP'],
 'NP ': [' APRO NP', ' AP N'],
 'V ': [' VERB', ' VERB', ' VERB', ' VERB', ' INFN'],
 'NProp ': [' NOUN'],
 'N ': [' NOUN', ' NOUN', ' NOUN', ' NOUN'],
 'AP ': ['', ' A'],
 'D ': ['', ' ADJF', ' ADJF', ' ADJF'],
 'APRO ': [' ADJF'],
 'A ': [' ADJF'],
 'SPRO ': [' NPRO']}

Здесь я хочу вручную переписать правила (благо их здесь немного)

In [32]:
nonterminal_grammar = """
S -> DP VP
DP -> D NP | NPRO |  
D -> Apro | 
NP -> AP NP | NOUN
AP -> ADJF
VP -> V DP | V VP | V
V -> VERB | INFN
"""

Переопределим функцию, которая добавляет терминалы в грамматику:

In [33]:
def get_terminal(word):
    tags = str(morph.parse(word)[0].tag).split(',')[:2]
    if tags[1].startswith('Apro'):
        tags = 'Apro'
    else:
        tags = tags[0]
    return tags

In [34]:
def add_terms_to_grammar(sentence, rules):
    new_words = []
    for word in sentence.lower().split():
        new_words.append(' -> '.join([get_terminal(word), "'{}'".format(word)]))
    rules = rules + '\n' + '\n'.join(new_words)
    new_grammar = nltk.CFG.fromstring(rules)
    parser = nltk.EarleyChartParser(new_grammar)
    return parser

И функцию, которая строит деревья:

In [35]:
def tree_all_new_words(sentence):
    parser = add_terms_to_grammar(sentence, nonterminal_grammar)
    print('Producted %d tree(s)'%len(list(parser.parse(sentence.lower().split()))))
    next(parser.parse(sentence.lower().split())).draw()

In [36]:
tree_all_new_words('Умный Вася поет какие-то странные песни')

Producted 1 tree(s)


In [37]:
tree_all_new_words('Кошка подавилась этой несчастной грушей')

Producted 1 tree(s)


In [38]:
tree_all_new_words('Вася поет песни')

Producted 1 tree(s)


Я считаю, что это победа!